In [1]:
import pandas as pd
from util import read_data, save, plt_encoding_error, error, normal

In [2]:
train = pd.read_csv("../common/data/processed/train.csv")
test_X = pd.read_csv("../common/data/processed/test.csv")
test_X.pop("id")
test_X = test_X.as_matrix()

train.pop("id")
target = train.pop("血糖")
features = train.columns.tolist()

train_X = train.as_matrix()
train_Y = target.as_matrix()


In [ ]:
# large_list = list()
# for i, y in enumerate(train_Y):
#     if y > 7:
#         large_list.append(i)

In [ ]:
large_list

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
train_X = PolynomialFeatures().fit_transform(train_X)

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_X, train_Y, test_size=0.2, random_state=1)

In [4]:
from sklearn.svm import SVR
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

param_grid = dict(C=[8, 10, 12, 50],
                  epsilon=[0.01,0.02 , 0.04],
                  gamma = [0.015, 0.02, 0.03]
                  )

clf = SVR(C=10, epsilon=0.02)

grid = GridSearchCV(cv=5, estimator=clf, n_jobs=-1, param_grid=param_grid, scoring='neg_mean_squared_error')
grid.fit(X_train, y_train)
print('Best parameters found by grid search are:', grid.best_params_, '  best_score: ', grid.best_score_)
y_pred = grid.predict(X_test)
# eval
error(y_test, y_pred)

Best parameters found by grid search are: {'C': 12, 'epsilon': 0.02, 'gamma': 0.02}   best_score:  -2.45676047591
1/2 Mean squared error: 1.054193


1.0541929148447271

In [ ]:
y_pred

In [ ]:
from sklearn.svm import SVR
clf = SVR(C=10, epsilon=0.02)

In [6]:
params = {
    'boosting': 'gbdt',
    'objective': 'regression',
    'metric': 'mse',
    'num_leaves': 31,
    'min_data_in_leaf': 20,
    'learning_rate': 0.02,
#     'lambda_l1':1,
#     'lambda_l2':0.2,
    'cat_smooth':10,
    'feature_fraction': 0.8,
    'bagging_freq': 5,
    'verbosity':-1
}

In [9]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.svm import SVR

X = train_X
Y = train_Y
N = 5
# kf = KFold(n_splits=N, shuffle=True, random_state=201801)
kf = KFold(n_splits=N, random_state=42)
result_mean = 0.0
i = 0
test_preds = np.zeros((test_X.shape[0], N))
for train_index, test_index in kf.split(X):
    training_features, training_target = X[train_index], Y[train_index]
    testing_features, testing_target = X[test_index], Y[test_index]
    
    lgb_train = lgb.Dataset(training_features, training_target)

    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=283)

    results = gbm.predict(testing_features)
    
    clf = SVR(C= 12,epsilon = 0.02, gamma=0.015)
    clf.fit(training_features, training_target)
    results_clf = clf.predict(testing_features)
    
    test_preds[:, i] = gbm.predict(test_X)
    i += 1
    aa = (1 * results +0 *results_clf)
    result_mean += np.round(mean_squared_error(testing_target, aa), 5)
    print(np.round(mean_squared_error(testing_target, aa), 5) / 2)
result_mean /= N
print("Mean squared error: %.5f" % (result_mean / 2))

# submission = pd.DataFrame({'pred':test_preds.mean(axis=1)})
# save(submission, 'tpop_kfold_True_201801_96643')

0.60756
0.95925
0.929345
1.448625
1.00972
Mean squared error: 0.99090


In [8]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error


# create dataset for lightgbm
# lgb_train = lgb.Dataset(X_train, y_train, feature_name = features)
# lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train, feature_name = features)

# train_all = lgb.Dataset(train_X, train_Y, feature_name = features)

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

train_all = lgb.Dataset(train_X, train_Y)

# specify your configurations as a dict

print('Start training...')
# train
gbm = lgb.train(params,
                   lgb_train,
                    num_boost_round=283,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=50)

print('Start predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
error(y_test, y_pred)

# online

# predict = gbm.predict(test_X, num_iteration=gbm.best_iteration)
# data1 = pd.DataFrame(predict)
# save
# save(data1, 'lgb')

# gbm_online = lgb.train(params,
#                 train_all,
#                 num_boost_round=280)
# # predict
# predict = gbm_online.predict(test_X, num_iteration=gbm_online.best_iteration)
# data1 = pd.DataFrame(predict)
# # save
# save(data1, 'lgb')

Start training...
[1]	valid_0's l2: 2.09597
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's l2: 2.08519
[3]	valid_0's l2: 2.07682
[4]	valid_0's l2: 2.06684
[5]	valid_0's l2: 2.05582
[6]	valid_0's l2: 2.04506
[7]	valid_0's l2: 2.0336
[8]	valid_0's l2: 2.02326
[9]	valid_0's l2: 2.01473
[10]	valid_0's l2: 2.00865
[11]	valid_0's l2: 1.9977
[12]	valid_0's l2: 1.99029
[13]	valid_0's l2: 1.98361
[14]	valid_0's l2: 1.97697
[15]	valid_0's l2: 1.96894
[16]	valid_0's l2: 1.96183
[17]	valid_0's l2: 1.95514
[18]	valid_0's l2: 1.94812
[19]	valid_0's l2: 1.93888
[20]	valid_0's l2: 1.93308
[21]	valid_0's l2: 1.92675
[22]	valid_0's l2: 1.92106
[23]	valid_0's l2: 1.91488
[24]	valid_0's l2: 1.90951
[25]	valid_0's l2: 1.90357
[26]	valid_0's l2: 1.89789
[27]	valid_0's l2: 1.89383
[28]	valid_0's l2: 1.88791
[29]	valid_0's l2: 1.88239
[30]	valid_0's l2: 1.87729
[31]	valid_0's l2: 1.87347
[32]	valid_0's l2: 1.8679
[33]	valid_0's l2: 1.86218
[34]	valid_0's l2: 1.85781
[35]	valid_0's

0.83340133402546202

In [ ]:
plt_encoding_error()
lgb.plot_importance(gbm)
plt.show()